In [ ]:
!pip install transformers accelerate evaluate datasets peft -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install triton

  Using cached triton-3.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
Using cached triton-3.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (209.5 MB)


In [ ]:
import transformers
import accelerate
import peft

print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")
print(f"PEFT version: {peft.__version__}")

Transformers version: 4.46.2
Accelerate version: 1.1.1
PEFT version: 0.13.2


In [ ]:
model_checkpoint = "facebook/deit-base-patch16-224"

In [ ]:
from datasets import load_dataset

dataset = load_dataset("food101", split="train[:5000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
labels = dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

'baklava'

In [ ]:
from transformers import AutoFeatureExtractor, ViTForImageClassification

In [ ]:
image_processor = AutoFeatureExtractor.from_pretrained('facebook/deit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('facebook/deit-base-patch16-224')

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
train_transforms = Compose(
    [
        RandomResizedCrop(image_processor.size["height"]),
        RandomHorizontalFlip(),
        ToTensor(),
        normalize,
    ]
)

val_transforms = Compose(
    [
        Resize(image_processor.size["height"]),
        CenterCrop(image_processor.size["height"]),
        ToTensor(),
        normalize,
    ]
)


def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [train_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch


def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [ ]:
splits = dataset.train_test_split(test_size=0.1)
train_ds = splits["train"]
val_ds = splits["test"]

In [ ]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [ ]:
print_trainable_parameters(model)

trainable params: 86567656 || all params: 86567656 || trainable%: 100.00


In [ ]:
for name, module in model.named_modules():
    print(name, ":", module)

 : ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in

In [ ]:
import re
# def extract_target_layers(model):
#     """
#     Extracts layers matching attention blocks like:
#     'encoder.layer.X.attention.attention.query',
#     'encoder.layer.X.attention.attention.key',
#     'encoder.layer.X.attention.attention.value'
#     """
#     # Adjust the pattern based on actual printed names
#     pattern = re.compile(r'vit\.encoder\.layer\.\d+\.attention\.attention\.(query|key|value)')

#     # Extract matching layers
#     target_layers = [name for name, _ in model.named_parameters() if pattern.search(name)]

#     return target_layers

# # Extract the target layers
# target_layers = extract_target_layers(model)
# print(target_layers)

def extract_target_layers(model):
    """
    Extracts layer names matching the pattern:
    'vit.encoder.layer.X.attention.attention.query'
    'vit.encoder.layer.X.attention.attention.key'
    'vit.encoder.layer.X.attention.attention.value'
    without the '.weight' or '.bias' suffixes.
    """
    # Define the regex pattern to match only the base layer names
    pattern = re.compile(r'(vit\.encoder\.layer\.\d+\.attention\.attention\.(query|key|value))')

    # Use a set to avoid duplicates
    target_layers = set()

    # Extract matching layers from model parameters
    for name, _ in model.named_parameters():
        match = pattern.match(name)
        if match:
            # Add only the base layer name (without '.weight' or '.bias')
            target_layers.add(match.group(1))

    # Convert set to list and return
    return list(target_layers)

# Extract the target layers
target_layers = extract_target_layers(model)

# Print the extracted layers
print(target_layers)

['vit.encoder.layer.0.attention.attention.query', 'vit.encoder.layer.8.attention.attention.key', 'vit.encoder.layer.5.attention.attention.query', 'vit.encoder.layer.2.attention.attention.key', 'vit.encoder.layer.11.attention.attention.value', 'vit.encoder.layer.2.attention.attention.value', 'vit.encoder.layer.10.attention.attention.value', 'vit.encoder.layer.8.attention.attention.query', 'vit.encoder.layer.10.attention.attention.query', 'vit.encoder.layer.5.attention.attention.value', 'vit.encoder.layer.8.attention.attention.value', 'vit.encoder.layer.4.attention.attention.value', 'vit.encoder.layer.6.attention.attention.value', 'vit.encoder.layer.9.attention.attention.key', 'vit.encoder.layer.4.attention.attention.query', 'vit.encoder.layer.3.attention.attention.query', 'vit.encoder.layer.9.attention.attention.value', 'vit.encoder.layer.11.attention.attention.query', 'vit.encoder.layer.3.attention.attention.value', 'vit.encoder.layer.9.attention.attention.query', 'vit.encoder.layer.4.

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules = target_layers,
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)

trainable params: 4307944 || all params: 90875600 || trainable%: 4.74


In [ ]:
import torch

In [ ]:
torch._dynamo.list_backends()

['cudagraphs', 'inductor', 'onnxrt', 'openxla', 'tvm']

In [ ]:
lora_model_c = torch.compile(lora_model, backend = 'inductor')

In [ ]:
import numpy as np
import evaluate

# Load the evaluation metrics
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    """Computes accuracy, precision, recall, and F1 score on a batch of predictions."""
    # Get the predictions and the true labels
    predictions = np.argmax(eval_pred.predictions, axis=1)
    references = eval_pred.label_ids

    # Compute each metric
    accuracy = accuracy_metric.compute(predictions=predictions, references=references)
    precision = precision_metric.compute(predictions=predictions, references=references, average='weighted')
    recall = recall_metric.compute(predictions=predictions, references=references, average='weighted')
    f1 = f1_metric.compute(predictions=predictions, references=references, average='weighted')

    # Return all metrics as a dictionary
    return {
        "accuracy": accuracy["accuracy"],
        "precision": precision["precision"],
        "recall": recall["recall"],
        "f1": f1["f1"]
    }

In [ ]:
import torch


def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
import torch
from transformers import TrainingArguments, Trainer
import logging

# Ensure the model is loaded on the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to GPU
# lora_model = lora_model.to(device)

lora_model_c = lora_model_c.to(device)

# Enable logging and tqdm
logging.basicConfig(level=logging.INFO)
from transformers.utils import logging as hf_logging
hf_logging.set_verbosity_info()


model_name = model_checkpoint.split("/")[-1]
batch_size = 128

args = TrainingArguments(
    f"{model_name}-finetuned-lora-food101",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-3,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    fp16=torch.cuda.is_available(),  # Use mixed precision only if GPU is available
    num_train_epochs=5,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    label_names=["labels"],
    disable_tqdm=False,  # Disable tqdm progress bar
)

trainer = Trainer(
    lora_model_c,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
<ipython-input-20-3bbaa87a992f>:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Using auto half precision backend


In [ ]:
import wandb

# Initialize wandb
wandb.init(
    project=f"{model_name}-finetuning-food101-attentionheads_torchCompile_LoRA",
    group="LoRA-vs-Base",
    name=f"{model_checkpoint}-run-{wandb.util.generate_id()}_Rank64",
    config=args
)

In [ ]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [ ]:
train_results = trainer.train()

***** Running training *****
  Num examples = 4,500
  Num Epochs = 5
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 4
  Total optimization steps = 45
  Number of trainable parameters = 4,307,944


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.273320,0.908000,0.909254,0.908000,0.907345
2,0.426700,0.245476,0.898000,0.896974,0.898000,0.893909
3,0.292300,0.260899,0.906000,0.906823,0.906000,0.904325
4,0.238400,0.211795,0.918000,0.918548,0.918000,0.917711
5,0.200500,0.199993,0.920000,0.920229,0.920000,0.919648



***** Running Evaluation *****
  Num examples = 500
  Batch size = 128
W1114 20:20:21.701000 10527 torch/_dynamo/convert_frame.py:1125] WON'T CONVERT forward /usr/local/lib/python3.10/dist-packages/transformers/models/vit/modeling_vit.py line 831 
W1114 20:20:21.701000 10527 torch/_dynamo/convert_frame.py:1125] due to: 
W1114 20:20:21.701000 10527 torch/_dynamo/convert_frame.py:1125] Traceback (most recent call last):
W1114 20:20:21.701000 10527 torch/_dynamo/convert_frame.py:1125]   File "/usr/local/lib/python3.10/dist-packages/torch/_dynamo/output_graph.py", line 1446, in _call_user_compiler
W1114 20:20:21.701000 10527 torch/_dynamo/convert_frame.py:1125]     compiled_fn = compiler_fn(gm, self.example_inputs())
W1114 20:20:21.701000 10527 torch/_dynamo/convert_frame.py:1125]   File "/usr/local/lib/python3.10/dist-packages/torch/_dynamo/repro/after_dynamo.py", line 129, in __call__
W1114 20:20:21.701000 10527 torch/_dynamo/convert_frame.py:1125]     compiled_gm = compiler_fn(gm, exam